# LOPF with coupling to heating sector

In this example three locations are optimised, each with an electric bus and a heating bus and corresponding loads. At each location the electric and heating buses are connected with heat pumps; heat can also be supplied to the heat bus with a boiler. The electric buses are connected with transmission lines and there are electrical generators at two of the nodes.

In [ ]:
%pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import pypsa

sns.set(rc={"figure.figsize": (9, 5)})

In [ ]:
network = pypsa.Network()

Add three buses of AC and heat carrier each

In [ ]:
for i in range(3):
    network.add("Bus", f"electric bus {i}", v_nom=20.0)
    network.add("Bus", f"heat bus {i}", carrier="heat")
network.buses

In [ ]:
network.buses["carrier"].value_counts()

Add three lines in a ring

In [ ]:
for i in range(3):
    network.add(
        "Line",
        f"line {i}",
        bus0=f"electric bus {i}",
        bus1=f"electric bus {(i + 1) % 3}",
        x=0.1,
        s_nom=1000,
    )
network.lines

Connect the electric to the heat buses with heat pumps with COP 3

In [ ]:
for i in range(3):
    network.add(
        "Link",
        f"heat pump {i}",
        bus0=f"electric bus {i}",
        bus1=f"heat bus {i}",
        p_nom=100,
        efficiency=3.0,
    )
network.links

Add carriers

In [ ]:
network.add("Carrier", "gas", co2_emissions=0.27)
network.add("Carrier", "biomass", co2_emissions=0.0)
network.carriers

Add a gas generator at bus 0, a biomass generator at bus 1 and a boiler at all heat buses

In [ ]:
network.add(
    "Generator",
    "gas generator",
    bus="electric bus 0",
    p_nom=100,
    marginal_cost=50,
    carrier="gas",
    efficiency=0.3,
)

network.add(
    "Generator",
    "biomass generator",
    bus="electric bus 1",
    p_nom=100,
    marginal_cost=100,
    efficiency=0.3,
    carrier="biomass",
)

for i in range(3):
    network.add(
        "Generator",
        f"boiler {i}",
        bus=f"heat bus {i}",
        p_nom=1000,
        efficiency=0.9,
        marginal_cost=20.0,
        carrier="gas",
    )

network.generators

Add electric loads and heat loads.

In [ ]:
for i in range(3):
    network.add(
        "Load",
        f"electric load {i}",
        bus=f"electric bus {i}",
        p_set=i * 10,
    )

for i in range(3):
    network.add(
        "Load",
        f"heat load {i}",
        bus=f"heat bus {i}",
        p_set=(3 - i) * 10,
    )

network.loads

We define a function for the LOPF

In [ ]:
def run_lopf():
    network.optimize()
    df = pd.concat(
        [
            network.generators_t.p.loc["now"],
            network.links_t.p0.loc["now"],
            network.loads_t.p.loc["now"],
        ],
        keys=["Generators", "Links", "Line"],
        names=["Component", "index"],
    ).reset_index(name="Production")

    sns.barplot(data=df, x="index", y="Production", hue="Component")
    plt.title(f"Objective: {network.objective}")
    plt.xticks(rotation=90)
    plt.tight_layout()

In [ ]:
run_lopf()

Now, rerun with marginal costs for the heat pump operation.

In [ ]:
network.links.marginal_cost = 10
run_lopf()

Finally, rerun with no CO2 emissions.

In [ ]:
network.add("GlobalConstraint", "co2_limit", sense="<=", constant=0.0)

run_lopf()